# Cortex Analyst徹底検証 #2 セットアップ

## 概要
- このNotebookは、Zennの記事：[Snowflake Cortex Analyst徹底検証 #2：JOIN編と検証から見えた設計上の考慮事項](https://zenn.dev/nttdata_tech/articles/305a605eac9f61)に対応します  
- このNotebookは、上記のZenn記事の環境設定を補助するために使用します。
- 各自のSnowflakeアカウント>Notebooksに、この.ipynbファイルをimportして使用してください
- 以降の項番は、Zennの記事に対応させています

## 2.1 環境準備

### ステップ１.基本設定
検証用のSCHEMAを作成します（DBは#1で使用したものを流用）

In [ ]:
USE ROLE SYSADMIN;
USE DATABASE CORTEX_LAB;
CREATE SCHEMA IF NOT EXISTS CORTEX_LAB.L3;
USE SCHEMA CORTEX_LAB.L3;

### ステップ2.テーブルの作成
ITEM_DIMテーブルを作成します

In [ ]:
CREATE OR REPLACE TABLE ITEM_DIM (
  ITEM_ID    NUMBER        PRIMARY KEY, 
  ITEM_NAME  STRING        NOT NULL,
  CATEGORY   STRING        NOT NULL, 
  BRAND      STRING                            
);

### ステップ3:サンプルデータの挿入

In [ ]:
INSERT INTO ITEM_DIM
WITH PARAMS AS (
  SELECT
    ARRAY_CONSTRUCT
      ('Cola','Tea','Coffee','Mineral Water','Juice','Energy Drink',
       'Chips','Nuts','Cookie','Crackers',
       'Chocolate','Cake','Ice Cream',
       'Salmon','Tuna','Shrimp','Cod','Mackerel',
       'Beef','Pork','Chicken',
       'Lettuce','Tomato','Onion','Potato','Carrot',
       'Banana','Apple','Orange','Grape',
       'Milk','Cheese','Yogurt','Butter',
       'Rice','Bread','Pasta','Flour','Sugar',
       'Other Item'
      ) AS ITEMS,
    ARRAY_CONSTRUCT
      ('Beverage','Beverage','Beverage','Beverage','Beverage','Beverage',
       'Snack','Snack','Snack','Snack',
       'Sweets','Sweets','Sweets',
       'Seafood','Seafood','Seafood','Seafood','Seafood',
       'Meat','Meat','Meat',
       'Vegetable','Vegetable','Vegetable','Vegetable','Vegetable',
       'Fruit','Fruit','Fruit','Fruit',
       'Dairy','Dairy','Dairy','Dairy',
       'Staple','Staple','Staple','Staple','Staple',
       'Other'
      ) AS CATS,
    ARRAY_CONSTRUCT('Acme','Nimbus','Polar','Orion','ZenFoods','Maple','Kite','Aurora') AS BRANDS
),
SEQ AS (
  SELECT SEQ8() AS IDX
  FROM TABLE(GENERATOR(ROWCOUNT => 40))
),
BIASED_BRAND AS (
  SELECT
    IDX,
    CASE
      WHEN RANDOM() < 0.30 THEN 'Acme'
      WHEN RANDOM() < 0.55 THEN 'Nimbus'
      ELSE GET((SELECT BRANDS FROM PARAMS), CAST(UNIFORM(2, 8, RANDOM()) AS INT))::STRING -- 2..7=Polar..Aurora
    END AS BRAND
  FROM SEQ
)
SELECT
  1000 + s.IDX                                                   AS ITEM_ID,
  GET(p.ITEMS, s.IDX)::STRING                                    AS ITEM_NAME,
  GET(p.CATS , s.IDX)::STRING                                    AS CATEGORY,
  b.BRAND                                                        AS BRAND
FROM SEQ s
CROSS JOIN PARAMS p
JOIN BIASED_BRAND b ON b.IDX = s.IDX
ORDER BY ITEM_ID;

In [ ]:
-- テーブルを確認
SELECT * FROM ITEM_DIM;

### ステップ4.テーブルの作成
REGION_DIMテーブルを作成します

In [ ]:
CREATE OR REPLACE TABLE REGION_DIM (
  REGION_ID   NUMBER       PRIMARY KEY,
  REGION_NAME STRING       NOT NULL
);

### ステップ5:サンプルデータの挿入

In [ ]:
INSERT INTO REGION_DIM (REGION_ID, REGION_NAME)
VALUES
  (10,  'Hokkaido'),
  (20,  'Tohoku'),
  (30,  'Kanto'),
  (40,  'Chubu'),
  (50,  'Kansai'),
  (60,  'Chugoku'),
  (70,  'Shikoku'),
  (80,  'Kyushu'),
  (90,  'Okinawa');

In [ ]:
-- テーブル確認
SELECT * FROM REGION_DIM;

### ステップ6.テーブルの作成
STORE_DIMテーブルを作成します

In [ ]:
CREATE OR REPLACE TABLE STORE_DIM (
  STORE_ID    NUMBER       PRIMARY KEY,   
  STORE_NAME  STRING       NOT NULL,      
  REGION_ID   NUMBER       NOT NULL,             
  STORE_TYPE  STRING,                               
  OPEN_DATE   DATE,                          
  CONSTRAINT FK_STORE_REGION
    FOREIGN KEY (REGION_ID) REFERENCES REGION_DIM(REGION_ID)
);

### ステップ7:サンプルデータの挿入

In [ ]:
INSERT INTO STORE_DIM (STORE_ID, STORE_NAME, REGION_ID, STORE_TYPE, OPEN_DATE)
WITH
SEQ AS (
  SELECT SEQ8() AS IDX
  FROM TABLE(GENERATOR(ROWCOUNT => 60))               -- 0..59
),
PARAMS AS (
  SELECT
    ARRAY_CONSTRUCT('Metro','Mall','Suburban','Outlet','Airport') AS TYPES
),
REGION_BUCKET AS (
  SELECT
    s.IDX,
    CASE
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 18 THEN 30  -- Kanto
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 30 THEN 50  -- Kansai
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 39 THEN 40  -- Chubu
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 46 THEN 80  -- Kyushu
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 51 THEN 20  -- Tohoku
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 54 THEN 10  -- Hokkaido
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 57 THEN 60  -- Chugoku
      WHEN NTILE(60) OVER (ORDER BY s.IDX) <= 59 THEN 70  -- Shikoku
      ELSE 90                                           -- Okinawa
    END AS REGION_ID
  FROM SEQ s
),
RAND AS (
  SELECT
    IDX,
    UNIFORM(-3650, 0, RANDOM()) AS DAY_SHIFT 
  FROM SEQ
)
SELECT
  100 + s.IDX                                                        AS STORE_ID, 
  'Store_' || LPAD(TO_VARCHAR(1 + s.IDX), 3, '0')                    AS STORE_NAME,
  rb.REGION_ID                                                       AS REGION_ID,
  GET(p.TYPES, MOD(ABS(HASH(TO_VARCHAR(100 + s.IDX))), ARRAY_SIZE(p.TYPES)))::STRING
    AS STORE_TYPE,
  DATEADD('day', r.DAY_SHIFT, CURRENT_DATE())                        AS OPEN_DATE
FROM SEQ s
JOIN REGION_BUCKET rb ON rb.IDX = s.IDX
JOIN RAND r          ON r.IDX = s.IDX
CROSS JOIN PARAMS p
ORDER BY STORE_ID;

In [ ]:
-- テーブル確認
SELECT * FROM STORE_DIM;

### ステップ8.テーブルの作成
SALES_L3テーブルを作成します

In [ ]:
CREATE OR REPLACE TABLE SALES_L3 (
  ORDER_ID   NUMBER         NOT NULL,  
  ORDER_DATE DATE           NOT NULL,       
  ITEM_ID    NUMBER         NOT NULL,         
  STORE_ID   NUMBER         NOT NULL,       
  QUANTITY   NUMBER(4)      NOT NULL,    
  PRICE      NUMBER(10,2)   NOT NULL,    
  CONSTRAINT FK_SALES_ITEM  FOREIGN KEY (ITEM_ID)  REFERENCES ITEM_DIM(ITEM_ID),
  CONSTRAINT FK_SALES_STORE FOREIGN KEY (STORE_ID) REFERENCES STORE_DIM(STORE_ID)
);

### ステップ9:サンプルデータの挿入

In [ ]:
INSERT INTO SALES_L3 (ORDER_ID, ORDER_DATE, ITEM_ID, STORE_ID, QUANTITY, PRICE)
WITH
BASE AS (
  SELECT SEQ8() AS SEQ
  FROM TABLE(GENERATOR(ROWCOUNT => 2000))
),
ITEMS AS (
  SELECT ITEM_ID, CATEGORY, ROW_NUMBER() OVER (ORDER BY ITEM_ID) AS RN, COUNT(*) OVER () AS CNT
  FROM ITEM_DIM
),
STORES AS (
  SELECT STORE_ID, ROW_NUMBER() OVER (ORDER BY STORE_ID) AS RN, COUNT(*) OVER () AS CNT
  FROM STORE_DIM
),
PICKED AS (
  SELECT
    b.SEQ,
    DATEADD('day', UNIFORM(0, 1096, RANDOM()), DATE('2023-01-01')) AS ORDER_DATE, 
    1 + MOD(UNIFORM(0, 1000000, RANDOM()), (SELECT MAX(CNT) FROM ITEMS))  AS RN_ITEM, 
    1 + MOD(UNIFORM(0, 1000000, RANDOM()), (SELECT MAX(CNT) FROM STORES)) AS RN_STORE, 
    1 + MOD(UNIFORM(0, 1000000, RANDOM()), 5) AS QUANTITY                         
  FROM BASE b
),
JOINED AS (
  SELECT
    p.SEQ,
    p.ORDER_DATE,
    i.ITEM_ID,
    i.CATEGORY,
    s.STORE_ID,
    p.QUANTITY
  FROM PICKED p
  JOIN ITEMS  i ON i.RN = p.RN_ITEM
  JOIN STORES s ON s.RN = p.RN_STORE
),
RANGES AS (
  SELECT
    j.*,
    CASE j.CATEGORY
      WHEN 'Seafood'   THEN 100000  -- 1000.00
      WHEN 'Meat'      THEN  80000  --  800.00
      WHEN 'Dairy'     THEN  12000  --  120.00
      WHEN 'Beverage'  THEN  10000  --  100.00
      WHEN 'Snack'     THEN  10000  --  100.00
      WHEN 'Sweets'    THEN  20000  --  200.00
      WHEN 'Vegetable' THEN   5000  --   50.00
      WHEN 'Fruit'     THEN   8000  --   80.00
      WHEN 'Staple'    THEN  20000  --  200.00
      ELSE                   10000  --  100.00
    END AS MIN_CENTS,
    CASE j.CATEGORY
      WHEN 'Seafood'   THEN 600000  -- 6000.00
      WHEN 'Meat'      THEN 480000  -- 4800.00
      WHEN 'Dairy'     THEN  72000  --  720.00
      WHEN 'Beverage'  THEN  50000  --  500.00
      WHEN 'Snack'     THEN  80000  --  800.00
      WHEN 'Sweets'    THEN 120000  -- 1200.00
      WHEN 'Vegetable' THEN  30000  --  300.00
      WHEN 'Fruit'     THEN  50000  --  500.00
      WHEN 'Staple'    THEN 200000  -- 2000.00
      ELSE                   100000 -- 1000.00
    END AS MAX_CENTS
  FROM JOINED j
),
PRICED AS (
  SELECT
    SEQ, ORDER_DATE, ITEM_ID, STORE_ID, QUANTITY,
    CAST(
      (MIN_CENTS + MOD(UNIFORM(0, 1000000, RANDOM()), (MAX_CENTS - MIN_CENTS + 1))) / 100.0
      AS NUMBER(10,2)
    ) AS UNIT_PRICE
  FROM RANGES
)
SELECT
  100000 + SEQ AS ORDER_ID,
  ORDER_DATE,
  ITEM_ID,
  STORE_ID,
  QUANTITY,
  UNIT_PRICE AS PRICE
FROM PRICED
ORDER BY ORDER_ID;


In [ ]:
-- テーブル確認
SELECT * FROM SALES_L3;

事前準備は以上です。各自、Cortex Analystの設定に移ってください。  

- Snowsight>AI&ML>Cortex Analyst
- ymlファイルはこのNotebookと同じリポジトリに格納しています。